In [ ]:
#Simplex Revisado
#Creando un problema
c=[-10 57 9 24 0 0 0] 
b=[0;0;1]
A=[0.5 -5.5 -2.5 9 1 0 0; 0.5 -1.5 -0.5 1 0 1 0; 1 0 0 0 0 0 1]

In [ ]:
#Ajuste de datos
function b_mayor_cero(A,b)
    for i=1:length(b)
        if b[i]<0
            b[i]=-b[i]
            A[i,:]=-A[i,:]
        end
    end
    return A,b
end

#Fase 1
#Construcción del problema auxiliar
function auxproblem(A,b,c)
    cf1=hcat(zeros(length(c))',ones(size(A,1))');
    I=eye(size(A,1),size(A,1));
    A=hcat(A,I);
        
    basicas=Vector(size(A,2)-size(I,1)+1:size(A,2));
    nobasicas=Vector(1:size(A,2)-size(I,1));
    B=A[:,size(A,2)-size(I,1)+1:size(A,2)];
    cB=cf1[size(A,2)-size(I,1)+1:size(A,2)]';
    N=A[:,1:size(A,2)-size(I,1)];
    cN=cf1[1:size(A,2)-size(I,1)]';
    
    return basicas,nobasicas,B,N,cB,cN
end

In [ ]:
#construimos base inicial
basicas = Array{Int16}()
nobasicas = Array{Int16}()
#vamos a dimensionar valores de calculo
B=Array{Float64}()
cB=Array{Float64}()
N=Array{Float64}()
cN=Array{Float64}()

#inicializamos B, cB, N, cN
A,b=b_mayor_cero(A,b);
A1=hcat(A,eye(size(A,1),size(A,1)))
A,b=b_mayor_cero(A,b);
basicas,nobasicas,B,N,cB,cN=auxproblem(A,b,c);
Binv=copy(B)
Nprima=copy(N)

In [ ]:
function iteracionSimplex(B,Binv,Nprima,cB,N,cN,b,fase)
    #iteración del simplex
    obj=cB*Binv*b
    println("obj: ",obj)
    p=cB*Binv
    cRed=cN-p*N
    #println("costos reducidos:",cRed)
    sol=Binv*b
    #println("solucion:",sol)
    if fase==1 && obj[1]==0
        return obj,-1,-1, true, false
    else
        v1,v2,opt,noacotado = selectPivote(cRed,Nprima,sol,Binv)
        return obj,v1,v2, opt, noacotado
    end
end

#seleccionar variable que entra y sale
function selectPivote(costosBasicos,matrizN,rhs,Binv)
    #println("matrizN: ",matrizN)
    #println("rhs: ",rhs)
    println("costosBasicos: ",costosBasicos)
    opt=false
    noacotado=false
    #Implementción regla de Bland
    #devuelve posicion con costo reducido negativo
    minCol=0
    for i in 1:length(costosBasicos)
        if costosBasicos[i]<0
            minCol=i
            break
        end
    end
    
    #Validación optimalidad
    if minCol==0
        println("Solucion Optima")
        opt=true
        return -1,-1,opt,noacotado
    end
    
    #Implementción regla lexicográfica
    u=matrizN[:,minCol]
    if all(u.<=0)
        noacotado=true
        println("El PL planteado es no acotado")
        return -1,-1,opt,noacotado
    end
    d=-rhs./u
    dm=d[(d.<=0)&(u.>0)]
    if all(dm.==0)==true
        newCol=1
    else
        teta=maximum(dm)
        teta_pos=find(x->x==teta,d)
        allv=vcat(basicas,nobasicas)
        A=hcat(Binv,matrizN)
        orden=sortperm(allv)
        A=A[:,orden]
        contador=1
        while length(teta_pos)>1
            if (all(A[teta_pos,contador].==A[teta_pos,contador][1]))
                contador=contador+1
            else
                delete=maximum(A[teta_pos,contador])
                delwhere=find(x->x.==delete,A[teta_pos,contador])
                teta_pos=teta_pos[setdiff(1:length(teta_pos),delwhere)]
            end
        end
        newCol=teta_pos[1]
    end

    println("entra columna ",minCol," sale ",newCol)
    return minCol,newCol,opt,noacotado
end

#Función para actualiza la base inversa con la nueva variable entrante
function actualiza_Binv(ent,sal,Binv,Nprima)
    Ag=hcat(Binv,Nprima[:,ent])
    for i=1:size(Binv,1)
        if i!=sal
            Ag[i,:]=Ag[i,:]-(Ag[i,size(Ag,2)]/Ag[sal,size(Ag,2)])*Ag[sal,:]
        else
            Ag[i,:]=Ag[i,:]./Ag[i,size(Ag,2)]
        end
    end
    return Ag[:,1:size(Binv,2)]
end    

In [ ]:
#Box para hacer una iteración
function ordenit(B,Binv,cB,N,Nprima,cN,b,basicas,nobasicas,fase)
    while true
        obj,variableIn,variableOut, opt, noacotado = iteracionSimplex(B,Binv,Nprima,cB,N,cN,b,fase)
        if variableIn != -1
            Binv=actualiza_Binv(variableIn,variableOut,Binv,Nprima)
            basicas[variableOut] , nobasicas[variableIn] = nobasicas[variableIn] , basicas[variableOut];
            cB[1,variableOut] , cN[1,variableIn] = cN[1,variableIn] , cB[1,variableOut]; 
            ordb=sortperm(basicas);
            ordnb=sortperm(nobasicas);
            cB=cB[ordb]';
            cN=cN[ordnb]';
            basicas=basicas[ordb];
            nobasicas=nobasicas[ordnb];
            Binv=Binv[ordb,:];
            N=A1[:,nobasicas];
            Nprima=Binv*A1[:,nobasicas];
            println("obj: ",obj[1]," cB: ", cB, " cN: ",cN," basicas: ",basicas," noBasicas ",nobasicas)
            println("##########---#########")
        else
            if noacotado==false
                println("obj: ",obj[1]," basicas: ",basicas," valores ",Binv*b)
                println("##########---#########")
                z=obj[1]
                return z,B,N,basicas,nobasicas,cB,cN,Binv,Nprima
            else
                println("obj: ", -Inf, "El problema es no acotado")
            end
        end
    end
end

In [ ]:
#Box para iterar en primera fase
z,B,N,basicas,nobasicas,cB,cN,Binv,Nprima=ordenit(B,Binv,cB,N,Nprima,cN,b,basicas,nobasicas,1)

In [ ]:
if z==0
    while length(find(x->x.>size(A,2),basicas))>0
        for i in length(basicas):-1:1
            if basicas[i]>size(A,2)
                newCol=i
                break
            end
        end
        
        if all(x->x.==0,(B^-1*A)[newCol,:])
            #En este if se eliminan filas que sean LI y se resuelve de nuevo el problema
            A=A[setdiff(1:size(A,1),basicas[newCol]-size(A,2)),:]
            b=b[setdiff(1:size(b,1),basicas[newCol]-size(A,2)),:]
            basicas = Array{Int16}()
            nobasicas = Array{Int16}()
            #vamos a dimensionar valores de calculo
            B=Array{Float64}()
            cB=Array{Float64}()
            N=Array{Float64}()
            cN=Array{Float64}()

            #inicializamos B, cB, N, cN
            basicas,nobasicas,B,N,cB,cN=auxproblem(A,b,c);
            #reconstruyendo la solución
            z,B,N,basicas,nobasicas,cB,cN,Binv,Nprima=ordenit(B,Binv,cB,N,Nprima,cN,b,basicas,nobasicas,1)
        #Driving out artificial variables
        else
            #Esta sección ayuda a eliminar una variable artificial de fase 1 que quedó en la base óptima de dicho problema
            println("hay que sacar variables artificiales de la base")
            minCol=0
            for i in 1:length(nobasicas)
                if nobasicas[i]<=size(A,2)
                    if (B^-1*A)[newCol,nobasicas[i]]!=0
                        if det(hcat(B[:,setdiff(1:size(B,2),newCol)],(B^-1*A)[:,nobasicas[i]]))!=0
                            minCol=i
                            break
                        end
                    end
                end
            end
            println("Sale la variable de la columna: ",newCol," e ingresa la variable de la columna: ",minCol)
            Binv=actualiza_Binv(minCol,newCol,Binv,Nprima)
            basicas[newCol] , nobasicas[minCol] = nobasicas[minCol] , basicas[newCol];
            cB[1,newCol] , cN[1,newCol] = cN[1,newCol] , cB[1,newCol]; 
            ordb=sortperm(basicas);
            ordnb=sortperm(nobasicas);
            cB=cB[ordb]';
            cN=cN[ordnb]';
            basicas=basicas[ordb];
            nobasicas=nobasicas[ordnb]; 
            Binv=Binv[ordb,:];
            N=A1[:,nobasicas];
            Nprima=Binv*A1[:,nobasicas];
        end
    end    
    nobasicas=setdiff(range(1,size(A,2)),basicas)
    N=A[:,nobasicas]
    cB=c[basicas]'
    cN=c[nobasicas]'
    z,B,N,basicas,nobasicas,cB,cN,Binv,Nprima=ordenit(B,Binv,cB,N,Nprima,cN,b,basicas,nobasicas,2)
else            
    println("El problema es infactible")
end